In [14]:
import pandas as pd 
from sklearn.metrics import r2_score

In [15]:
RSM_pred = pd.read_excel('FIT_RSM.xlsx')
DF_DOE = pd.read_excel('DATI_DOE_R81_ULT.xlsx')
RSM_pred


,RUN,FITS_S_MAX,FITS_MY,FITS_AM
0,1,23.612381,1119.133661,0.026916
1,1,23.612381,1119.133661,0.026916
2,1,23.612381,1119.133661,0.026916
3,1,23.612381,1119.133661,0.026916
4,2,18.348198,978.808271,0.025913
...,...,...,...,...
319,80,26.949381,1240.295349,0.025245
320,81,11.198968,854.003142,0.015778
321,81,11.198968,854.003142,0.015778
322,81,11.198968,854.003142,0.015778


In [16]:
Actual_target  = DF_DOE.iloc[:,[0, 6, 7, 8]] 
Actual_target

,RUN,UTS,E,εd
0,1,13.813450,877.215792,0.022443
1,1,14.492491,913.234170,0.022406
2,1,12.129329,835.553666,0.019148
3,1,14.126136,887.405452,0.022959
4,2,12.374963,777.422107,0.019813
...,...,...,...,...
319,80,35.907820,1256.351635,0.024638
320,81,10.254925,734.141293,0.017783
321,81,10.790767,905.172547,0.016165
322,81,11.066774,925.381347,0.013322


In [17]:
#R^2 value RSM
R_UTS = r2_score(Actual_target.iloc[:,1], RSM_pred.iloc[:,1])
R_E =r2_score(Actual_target.iloc[:,2], RSM_pred.iloc[:,2])
R_ed =r2_score(Actual_target.iloc[:,3], RSM_pred.iloc[:,3])

In [ ]:
#R^2 value UTS for pattern
UTS_lines
UTS_grid
UTS_cubic

0.5180110659567521

In [21]:
DF_DOE


,RUN,PROVINO,Pattern,Temperature,Speed,Thickness,UTS,E,εd
0,1,2,Grid,230,10,0.1,13.813450,877.215792,0.022443
1,1,3,Grid,230,10,0.1,14.492491,913.234170,0.022406
2,1,4,Grid,230,10,0.1,12.129329,835.553666,0.019148
3,1,5,Grid,230,10,0.1,14.126136,887.405452,0.022959
4,2,1,Grid,250,50,0.2,12.374963,777.422107,0.019813
...,...,...,...,...,...,...,...,...,...
319,80,4,Lines,230,10,0.2,35.907820,1256.351635,0.024638
320,81,1,Cubic,230,50,0.4,10.254925,734.141293,0.017783
321,81,2,Cubic,230,50,0.4,10.790767,905.172547,0.016165
322,81,3,Cubic,230,50,0.4,11.066774,925.381347,0.013322
